# BCI Python Project

Converting BCI project from BE521

### 1. Load in subject data

3 files loaded in from BCI Competition Website. Dataset 4.   

    1. sub1_comp.mat
    2. sub2_comp.mat
    3. sub3_comp.mat

Each file contains train_data, test_data, train_dg 

In [ ]:
import scipy.io as sio 
import numpy as np
from scipy import signal
from matplotlib import pyplot as plt


In [ ]:
sub1 = sio.loadmat('BCICIV_4_mat/sub1_comp.mat')

The cell above loads displays the output of the mat file. It loads as a dictionary and we can access the training data with the code below:

In [ ]:
train1 = sub1['train_data']
train1.shape

## A Simple Method  

1. Use a moving window 100 ms in length with 50 ms overlap and extract the same 6 features over each of the channels. Features were:  
    1. Average time-domain voltage  
    2. Average Frequency-domain magnitude in   
       5-15 Hz    
    3. 20-25 Hz  
    4. 75-115 Hz  
    5. 125-160 Hz  
    6. 160-175 Hz

2. Downsample dataglove traces so that each sample was separated by 50 ms, to keep them on the same time scale as the features

3. Used a linear regression to predict downampled finger flextion from all the EEG features from the previous 3 time windows (150 ms lag)

4. interpolate the prediction using a cubic spline back up to the riginal 1000 Hz sampling frequency, making sure that the first and last points in the data interpolaion were values we know. The interpolation was zero-padded at the beginning and end to time-align with the original flexion trace

First let's take a look at some plots of the training data and the data glove traces

Use a moving window to calculate the 6 features above. 

In [ ]:
# Calculate moving window average
channel1 = train1[:,4]
window_length = 100 
window = np.ones([window_length,])
avg_channel1 = np.convolve(channel1, window/window_length, 'valid')  #  moving window average

In [ ]:
# Creating a time vector
t = np.arange(50,400000,50)

In [ ]:
plt.plot(channel1[0:500])
plt.plot(t[0:10],avg_channel1[0:500:50])
plt.show()

In [ ]:
fs = 1000
stride = 50 #steps 50 samples at a time for 100 sample window (50% overlap)
X = channel1
plt.figure(1)
plt.subplot(2,1,1)
plt.plot(X)                             #plot first 5000 samples of single channel
plt.plot(t, avg_channel1[::50])  #plot moving window average 
plt.xlabel('samples')
plt.ylabel('ECoG signal')
plt.subplot(2,1,2)
plt.plot(X[:5000])
plt.plot(t[0:100], avg_channel1[:5000:50])
plt.xlabel('samples')
plt.ylabel('ECoG signal')
plt.show()


In [ ]:
X = channel1
f, t, Sxx = signal.spectrogram(X[0:5000], fs, nperseg=100, noverlap=50, nfft=128)


In [ ]:
# figure plotting taking a long time. ECOG signals are already bandpassed filtered up to 200 Hz so no need to plot that data
f_bandpassed = f[f<200]
nbands = f_bandpassed.shape[0]

In [ ]:
plt.figure(1)
plt.pcolormesh(t,f[f<200], Sxx[:nbands, :])
plt.ylabel('Frequency (Hz')
plt.xlabel('Time (sec)')
plt.show()


### Next Steps
- generate the 6 features for a single channel 
- check dimensions 
- generate all 6 features for 1 subject
- downsample data glove trace
- linear regression
- interpolate back to 1000 hz


In [ ]:
band1 = np.mean(Sxx[(f>5) & (f<15),:], axis=0)

fbands = np.array([[5,15],[20,25],[75,115],[125,160],[160,175]])
fbands



In [ ]:
np.arange(5)